In [ ]:
# -*- coding: utf-8 -*-

import arcpy


class Toolbox(object):
    def __init__(self):
        """Define the toolbox (the name of the toolbox is the name of the
        .pyt file)."""
        self.label = "My First Toolbox"
        self.alias = "toolbox"

        # List of tool classes associated with this toolbox
        self.tools = [HMTool, DMSTool, BLTool, HWATool, SATool]


class HMTool(object):
    def __init__(self):
        """Define the tool (tool name is the name of the class)."""
        self.label = "Health Monitor Tool"
        self.description = "Please provide your height and weight info to get feedback "
        self.canRunInBackground = False

    def getParameterInfo(self):
        """Define parameter definitions"""
        param0 = arcpy.Parameter(displayName="Weight in lbs", name="wt", datatype="GPString", parameterType="Required", direction="Input")
        param1 = arcpy.Parameter(displayName="Height in ft", name="ft", datatype="GPString", parameterType="Required", direction="Input")
        params = [param0, param1]
        return params

    def isLicensed(self):
        """Set whether tool is licensed to execute."""
        return True

    def updateParameters(self, parameters):
        """Modify the values and properties of parameters before internal
        validation is performed.  This method is called whenever a parameter
        has been changed."""
        return

    def updateMessages(self, parameters):
        """Modify the messages created by internal validation for each tool
        parameter.  This method is called after internal validation."""
        return

    def execute(self, parameters, messages):
        """The source code of the tool."""
        weight = float(parameters[0].valueAsText)
        height = float(parameters[1].valueAsText)
        sw = (height*30.48-105)/0.454
        if weight>sw*1.2:
            arcpy.AddMessage("Need urgent weight control.")
        elif weight<sw*0.8:
            arcpy.AddMessage("Need to eat more - too slim is also dangours.")
        elif weight<sw*0.9 and weight>sw*0.8:
            arcpy.AddMessage("You are healthy, but can still enjoy a little more food.")
        elif weight<sw*1.2 and weight>sw*1.1:
            arcpy.AddMessage("Not urgent, but can consider less food and more sports.")
        else:
            arcpy.AddMessage("Your weight is within the normal range.")
        #print("Your standard weight is %f" % sw)
        return

    def postExecute(self, parameters):
        """This method takes place after outputs are processed and
        added to the display."""
        return

class DMSTool(object):
    def __init__(self):
        """Define the tool (tool name is the name of the class)."""
        self.label = "Decimal Degree Converter Tool"
        self.description = "To convert dd:mm:ss value to decimal degrees"
        self.canRunInBackground = False

    def getParameterInfo(self):
        """Define parameter definitions"""
        param0 = arcpy.Parameter(displayName="Degrees", name="dd", datatype="GPLong", parameterType="Required", direction="Input")
        param0.filter.type = "Range"
        param0.filter.list = [0,180]
        param1 = arcpy.Parameter(displayName="Minutes", name="mm", datatype="GPLong", parameterType="Required", direction="Input")
        param1.filter.type = "Range"
        param1.filter.list = [0,60]
        param2 = arcpy.Parameter(displayName="Seconds", name="ss", datatype="GPLong", parameterType="Required", direction="Input")
        param2.filter.type = "Range"
        param2.filter.list = [0,60]
        params = [param0, param1, param2]
        return params

    def isLicensed(self):
        """Set whether tool is licensed to execute."""
        return True

    def updateParameters(self, parameters):
        """Modify the values and properties of parameters before internal
        validation is performed.  This method is called whenever a parameter
        has been changed."""
        return

    def updateMessages(self, parameters):
        """Modify the messages created by internal validation for each tool
        parameter.  This method is called after internal validation."""
        return

    def execute(self, parameters, messages):
        """The source code of the tool."""
        ddmmssString = raw_input("Please input a ddmmss value in the format dd:mm:ss.\n")
        ddmmssList = ddmmssString.split(":")
        dd = float(ddmmssList[0])
        mm = float(ddmmssList[1])
        ss = float(ddmmssList[2])
        decimalValue = dd + mm/60 + ss/3600
        arcpy.AddMessage ("The decimal value is "+ str(decimalValue))
        return

    def postExecute(self, parameters):
        """This method takes place after outputs are processed and
        added to the display."""
        return

class BLTool(object):
    def __init__(self):
        """Define the tool (tool name is the name of the class)."""
        self.label = "Find Best Location Tool"
        self.description = ""
        self.canRunInBackground = False

    def getParameterInfo(self):
        """Define parameter definitions"""
        param0 = arcpy.Parameter(displayName="Pick a city file:", name="cityFile", datatype="DETextfile", parameterType="Required", direction="Input")
        param1 = arcpy.Parameter(displayName="Pick a location file:", name="locationFile", datatype="GPString", parameterType="Required", direction="Input")
        params = [param0, param1]
        return params

    def isLicensed(self):
        """Set whether tool is licensed to execute."""
        return True

    def updateParameters(self, parameters):
        """Modify the values and properties of parameters before internal
        validation is performed.  This method is called whenever a parameter
        has been changed."""
        return

    def updateMessages(self, parameters):
        """Modify the messages created by internal validation for each tool
        parameter.  This method is called after internal validation."""
        return

    def execute(self, parameters, messages):
        """The source code of the tool."""
        cityFile = open(parameters[0].valueAsText,"r")
        locationFile = open(parameters[1].valueAsText,"r")
        cityLines = cityFile.readlines()[1:]
        locationLines = locationFile.readlines()[1:]
        locationList = []
        for aLocation in locationLines:
            sumDistance = 0
            locationFlds = aLocation.split(",")
            locationX = float(locationFlds[1])
            locationY = float(locationFlds[2])
            for aCity in cityLines:
                cityFlds = aCity.split(",")
                cityX = float(cityFlds[1])
                cityY = float(cityFlds[2])
                dist = ((locationX-cityX)**2+(locationY-cityY)**2)**.5
                sumDistance += dist
            locationList.append((locationFlds[0], sumDistance))
        sortedLocationList = sorted(locationList, key = lambda locationRec:locationRec[1])
        arcpy.AddMessage("The best location id is %s" % sortedLocationList[0][0])
        arcpy.AddMessage("The best average distance is " + str(sortedLocationList[0][1]/(5240*len(cityLines))))
        cityFile.close()
        locationFile.close()
        return

    def postExecute(self, parameters):
        """This method takes place after outputs are processed and
        added to the display."""
        return

class HWATool(object):
    def __init__(self):
        """Define the tool (tool name is the name of the class)."""
        self.label = "Highway Population Analysis Tool"
        self.description = ""
        self.canRunInBackground = False

    def getParameterInfo(self):
        """Define parameter definitions"""
        hwList = ["I10", "I20", "I40", "I60", "I70", "I80", "I85", "I90", "I95"]
        param0 = arcpy.Parameter(displayName="Interstate", name="intFC", datatype="DEFeatureClass", parameterType="Required", direction="Input")
        params = None
        return params

    def isLicensed(self):
        """Set whether tool is licensed to execute."""
        return True

    def updateParameters(self, parameters):
        """Modify the values and properties of parameters before internal
        validation is performed.  This method is called whenever a parameter
        has been changed."""
        return

    def updateMessages(self, parameters):
        """Modify the messages created by internal validation for each tool
        parameter.  This method is called after internal validation."""
        return

    def execute(self, parameters, messages):
        """The source code of the tool."""
        return

    def postExecute(self, parameters):
        """This method takes place after outputs are processed and
        added to the display."""
        return

class SATool(object):
    def __init__(self):
        """Define the tool (tool name is the name of the class)."""
        self.label = "Suitability Analysis Tool"
        self.description = ""
        self.canRunInBackground = False

    def getParameterInfo(self):
        """Define parameter definitions"""
        params = None
        return params

    def isLicensed(self):
        """Set whether tool is licensed to execute."""
        return True

    def updateParameters(self, parameters):
        """Modify the values and properties of parameters before internal
        validation is performed.  This method is called whenever a parameter
        has been changed."""
        return

    def updateMessages(self, parameters):
        """Modify the messages created by internal validation for each tool
        parameter.  This method is called after internal validation."""
        return

    def execute(self, parameters, messages):
        """The source code of the tool."""
        return

    def postExecute(self, parameters):
        """This method takes place after outputs are processed and
        added to the display."""
        return